In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [22]:
df=pd.read_csv('items.csv')

In [12]:
df.columns

Index(['series', 'identifier_swisscollections', 'subject_form',
       'library_call_number', 'resource_type',
       'date_of_publication_or_production_standardised', 'place_standardised',
       'physical_description', 'language', 'title'],
      dtype='object')

In [13]:
df.head()

,series,identifier_swisscollections,subject_form,library_call_number,resource_type,date_of_publication_or_production_standardised,place_standardised,physical_description,language,title
0,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224305501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1127","['Brief', 'Autograph', 'Archivmaterial / Archi...",18060815,"{'name': 'Kopenhagen', 'identifier': ['(DE-588...","['2 Bl. (2 S. beschrieben) ; 19,5 x 12,5 cm']",['Deutsch'],Brief an Breitkopf und Härtel
1,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224905501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17940115,"{'name': 'Zürich', 'identifier': ['(DE-588)406...","['2 Bl. (4 S. beschrieben) ; 22,5 x 19 cm']",['Deutsch'],Brief an [Gottlieb Hufeland]
2,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522225205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1121","['Brief', 'Autograph', 'Archivmaterial / Archi...",18070822,"{'name': 'Nürnberg', 'identifier': ['(DE-588)4...",['1 Bl. (2 S. beschrieben) ; 18 x 12 cm'],['Deutsch'],Brief an Hofrat
3,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17960601,"{'name': 'Mannheim', 'identifier': ['(DE-588)4...","['2 Bl. (1 S. beschrieben) ; 18,5 x 11,5 cm']",['Deutsch'],Brief an Unbekannt
4,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227605501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",18061213,"{'name': 'Berlin', 'identifier': ['(DE-588)400...",['1 Bl. (2 S. beschrieben) ; 23 x 19 cm'],['Deutsch'],Brief an [Christian Gottlob] Voigt


In [23]:
df.found_data=pd.to_datetime(df.date_of_publication_or_production_standardised, errors='coerce')

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_35104\2702049369.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.found_data=pd.to_datetime(df.date_of_publication_or_production_standardised, errors='coerce')


In [29]:
df

,series,identifier_swisscollections,subject_form,library_call_number,resource_type,date_of_publication_or_production_standardised,place_standardised,physical_description,language,title
0,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224305501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1127","['Brief', 'Autograph', 'Archivmaterial / Archi...",18060815,"{'name': 'Kopenhagen', 'identifier': ['(DE-588...","['2 Bl. (2 S. beschrieben) ; 19,5 x 12,5 cm']",['Deutsch'],Brief an Breitkopf und Härtel
1,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224905501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17940115,"{'name': 'Zürich', 'identifier': ['(DE-588)406...","['2 Bl. (4 S. beschrieben) ; 22,5 x 19 cm']",['Deutsch'],Brief an [Gottlieb Hufeland]
2,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522225205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1121","['Brief', 'Autograph', 'Archivmaterial / Archi...",18070822,"{'name': 'Nürnberg', 'identifier': ['(DE-588)4...",['1 Bl. (2 S. beschrieben) ; 18 x 12 cm'],['Deutsch'],Brief an Hofrat
3,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17960601,"{'name': 'Mannheim', 'identifier': ['(DE-588)4...","['2 Bl. (1 S. beschrieben) ; 18,5 x 11,5 cm']",['Deutsch'],Brief an Unbekannt
4,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227605501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",18061213,"{'name': 'Berlin', 'identifier': ['(DE-588)400...",['1 Bl. (2 S. beschrieben) ; 23 x 19 cm'],['Deutsch'],Brief an [Christian Gottlob] Voigt
...,...,...,...,...,...,...,...,...,...,...
100012,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446014805501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 454-459","['Brief', 'Archivmaterial / Dossier']",18591114/18600731,"{'name': 'Sankt Petersburg', 'identifier': ['(...",['8 Bl.'],['Deutsch'],6 Briefe an Gustav Teichmüller
100013,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446015605501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 422a-423","['Brief', 'Archivmaterial / Dossier']",18641007/18690830,{'name': 'Joala'},['8 Bl.'],['Deutsch'],3 Briefe an Gustav Teichmüller
100014,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446017505501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 96-97","['Brief', 'Archivmaterial / Dossier']",18600325/18670217,{'name': 'Plön'},['2 Bl.'],['Deutsch'],2 Briefe an Gustav Teichmüller
100015,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446017805501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 93",['Brief'],1857,"{'name': 'Tartu', 'identifier': ['(DE-588)1029...",['1 Bl. ; 13 x 21 cm'],['Deutsch'],Brief an Behm


In [34]:
def find_date_of_publication(x):
    try:
        return int(x[:4])
    except:
        print(x)
        return None


In [35]:
df.found_data=df.date_of_publication_or_production_standardised.apply(find_date_of_publication)

März 1796
711, ca zwischen 1930-1972
vor dem 1 Mai 1988/19980322
711, ca zwischen 1930-1972
vor dem 1 Mai 1988/19980322
vor dem 1 Mai 1988/19980322
ca 1917/1918
711, ca zwischen 1930-1972
711, ca zwischen 1930-1972


In [36]:
df.found_data.isna().sum()

9